<a href="https://colab.research.google.com/github/uss96/StudyGroup/blob/master/Project_%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D_(%EC%8B%A4%EC%8A%B5)%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 환경설정

In [1]:
project_name = "영화리뷰 감정 분석" # 프로젝트 이름
class_info = "3차수 A반" # 차수 정보
email = "uiseok@sk.com" # 회사 이메일(본인의 회사 이메일 입력)

## 모듈 설치 및 데이터 다운로드

In [2]:
!pip install --upgrade mySUNI -q

import mySUNI
from mySUNI import cds, utils
import urllib.request
import zipfile


project = cds.Project(project_name, class_info, email)
project.edu_name = 'SUNI_딥러닝'

# 이메일 설정 확인
if not project.email:
    raise print("[이메일이 누락되었습니다] 이메일을 설정해 주세요↑↑↑")


print('===' * 15)
print(f'패키지버전: {mySUNI.__version__}')
print('===' * 15)
print(f'프로젝트명: {project_name}\n차수 정보 : {class_info}\n이  메  일: {email}')
print('===' * 15)

# 데이터셋 다운로드
cds.download_project(project_name, class_info, email)


프로젝트: 영화리뷰 감정 분석
파일 목록

submission.csv
- data/영화리뷰 감정 분석/submission.csv

train.csv
- data/영화리뷰 감정 분석/train.csv

test.csv
- data/영화리뷰 감정 분석/test.csv



## 필요한 모듈 import 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import tensorflow as tf


pd.set_option('display.max_colwidth', None)

# Data 경로 설정
DATA_DIR = 'data'

# 경고 무시
warnings.filterwarnings('ignore')

SEED=123
np.random.seed(SEED)
tf.random.set_seed(SEED)

## 데이터 로드

### 1. 데이터셋 로드

In [4]:
# train 데이터셋 로드 (train.csv)
train = pd.read_csv(os.path.join(DATA_DIR, project_name, 'train.csv'))
                                 
# test 데이터셋 로드 (test.csv)
test = pd.read_csv(os.path.join(DATA_DIR, project_name, 'test.csv'))

In [5]:
train.head()

,document,label
0,코미디 영화 네요 보면서 많이 웃었어요,1
1,역시 일본인들의 일상현실생활이군 개막장국가답다,1
2,완전 잼없고 내용도 없음. 보지마삼,0
3,왠만해선 좋게 봐주려고 했는데 내 와이파이가 이딴 쓰레기 영화를 보기 위해 한 몸 다 받쳐 다운로드를 했다는 것에 도저히 참을 수가 없다. 스토리도 작화도 연출도 이도저도 아닌 최악의 영화.,0
4,멋져멋져..........,1


In [6]:
test.head()

,document
0,미야자키 아오이 전도연 닮았다고 생각되면 공감 눌러
1,너무 감동적이에요..이거보고 하치가 주인이 죽인지도 모르고 늙어가며 추운곳에서 기다리는 모습이 너무 안타깝네요...
2,역시 빠순이의 점수 대단해
3,슬픈사랑의 노래... 분위기가 인상적이었던 영화...
4,학교에서 일어나는 내용이 좀 재미있을 뿐 너무 지루해서 끝까지 보기 힘든영화. 재밌는 소재를 갖고 이렇게 못만들다니.


### 한글 형태소 분석기 로드

[도큐먼트 링크](https://konlpy.org/ko/latest/index.html)

아래 Cell을 실행하여 설치

In [7]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 453 kB 70.2 MB/s 


In [8]:
from konlpy.tag import Okt

# 형태소 분석기 로드
okt = Okt()

In [9]:
# morphs 적용시
okt.morphs('안녕하세요? 반갑습니다. 저는 파이썬을 사랑합니다')

['안녕하세요', '?', '반갑습니다', '.', '저', '는', '파이썬', '을', '사랑', '합니다']

In [10]:
# pos 적용시
okt.pos('안녕하세요? 반갑습니다. 저는 파이썬을 사랑합니다')

[('안녕하세요', 'Adjective'),
 ('?', 'Punctuation'),
 ('반갑습니다', 'Adjective'),
 ('.', 'Punctuation'),
 ('저', 'Noun'),
 ('는', 'Josa'),
 ('파이썬', 'Noun'),
 ('을', 'Josa'),
 ('사랑', 'Noun'),
 ('합니다', 'Verb')]

In [11]:
# 명사만 추출시
[word for word, tag in okt.pos('안녕하세요? 반갑습니다. 저는 파이썬을 사랑합니다') if tag == 'Noun']

['저', '파이썬', '사랑']

In [12]:
# 조사는 제외시
[word for word, tag in okt.pos('안녕하세요? 반갑습니다. 저는 파이썬을 사랑합니다') if tag != 'Josa']

['안녕하세요', '?', '반갑습니다', '.', '저', '파이썬', '사랑', '합니다']

In [ ]:
# 조사, 기호는 제외시
[word for word, tag in okt.pos('안녕하세요? 반갑습니다. 저는 파이썬을 사랑합니다') if not tag in ['Josa', 'Punctuation']]

['안녕하세요', '반갑습니다', '저', '파이썬', '사랑', '합니다']

In [ ]:
# 한글, 영어, 숫자만 추출시
import re

sample = "abcdefㄱㄴㄷㄹㅁㅂ가나다라마바사12345[]{}().,!?'`~;:"

re.sub('[^A-Za-z0-9가-힣]', '', sample)

'abcdef가나다라마바사12345'

In [ ]:
s = ['안녕하세요', '반갑습니다.', '저', '파이썬']
stopwords = ['파이썬', '안녕하세요'] # 특정 문장 제거

[w for w in s if w not in stopwords]

['반갑습니다.', '저']

### 전처리 함수 형태로 생성

In [13]:
stopword = ['영화', '들', '이', '적', '점']

def preprocessing(x):
    # 형태소 분석기
    x = [word for word, tag in okt.pos(x) if not tag in ['Josa', 'Punctuation']]
    x = [w for w in x if w not in stopword]
    # 기타 문장 전처리기 코드
    # 코드 입력
    
    return ' '.join(x)

### 전처리기 apply

In [14]:
train['document'].iloc[:20].apply(preprocessing) #일부만 적용시켜서 간 보기

0                                                                                코미디 네요 보면서 많이 웃었어요
1                                                                    역시 일본인 일상 현 실생활 이군 개막 장 국가 답 다
2                                                                               완전 잼 없고 내용 없음 보지마 삼
3     왠만해 선 좋게 봐주려고 했는데 내 와이파이 이딴 쓰레기 보기 위해 한 몸 다 받쳐 다운로드 했다는 것 도저히 참을 수가 없다 스토리 작화 연출 이도 저 도 아닌 최악
4                                                                                             멋져 멋져
5                                                                                    단 한마디 평가 하겠다 잼
6                                                                                         스물 다가는 시간
7                                                                                  완전 재미없음 반전 무슨 반전
8                                                                          cg 없이 한순간 질린 장면 없 엇 던 sf
9                                                                        파라 노말 엑 티비 티 줄 알았던데 잘못 알았다


In [15]:
train_sentences = train['document'].apply(preprocessing)
test_sentences = test['document'].apply(preprocessing)

### 데이터셋 분할

In [16]:
from sklearn.model_selection import train_test_split

SEED = 123

x_train, x_test, y_train, y_test = train_test_split(train_sentences, train['label'], 
                                                                            stratify=train['label'],
                                                                            test_size = 0.2, 
                                                                            random_state = SEED)

In [17]:
x_train.shape, x_test.shape

((8000,), (2000,))

### tensorflow 관련 모듈 import 

In [62]:
# Module Import
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint



### 단어 사전 정의

In [63]:
# 단어사전의 개수, OOV 토큰 지정
vocab_size = 10000 # 코드입력
oov_token = "<00V>" # 코드입력

In [64]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>') # 코드입력

In [65]:
# 단어사전 생성
tokenizer.fit_on_texts(x_train)# 코드입력

In [66]:
for key, value in tokenizer.word_index.items():
    print('{}  \t======>\t {}'.format(key, value))
    if value == 25:
        break

<OOV>  	======>	 1
너무  	======>	 2
정말  	======>	 3
진짜  	======>	 4
것  	======>	 5
그  	======>	 6
내  	======>	 7
평점  	======>	 8
생각  	======>	 9
잘  	======>	 10
연기  	======>	 11
스토리  	======>	 12
최고  	======>	 13
이런  	======>	 14
왜  	======>	 15
안  	======>	 16
하는  	======>	 17
보고  	======>	 18
한  	======>	 19
감동  	======>	 20
드라마  	======>	 21
말  	======>	 22
때  	======>	 23
다  	======>	 24
사람  	======>	 25


In [67]:
len(tokenizer.word_index)

19553

In [68]:
word_index = tokenizer.word_index

### 시퀀스 변환

In [69]:
# 분할한 x_train, x_test 시퀀스 변환
x_train_sequences = tokenizer.texts_to_sequences(x_train)
x_test_sequences = tokenizer.texts_to_sequences(x_test)

# test 데이터셋 변환
test_sequences = tokenizer.texts_to_sequences(test_sentences)

In [70]:
# 한 문장의 최대 단어 숫자
max_length = 40 # 코드입력

# 잘라낼 문장의 위치
trunc_type = 'post' # 코드입력

# 채워줄 문장의 위치
padding_type = 'post' # 코드입력

### padding 적용

In [71]:
x_train_padded = pad_sequences(x_train_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type) # 코드입력
x_test_padded = pad_sequences(x_test_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)# 코드입력
test_padded = pad_sequences(test_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)# 코드입력

In [72]:
train_labels = np.array(y_train)
test_labels = np.array(y_test)

In [73]:
embedding_dim = 16

### 모델 생성

In [74]:
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(64)),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(16, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

### 모델 요약

In [75]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 40, 16)            160000    
                                                                 
 bidirectional_16 (Bidirecti  (None, 40, 128)          41472     
 onal)                                                           
                                                                 
 bidirectional_17 (Bidirecti  (None, 40, 128)          98816     
 onal)                                                           
                                                                 
 bidirectional_18 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dense_19 (Dense)            (None, 32)                4128      
                                                      

### 모델 컴파일

In [82]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0004)
 
 #코드입력
model.compile(optimizer, loss='binary_crossentropy', metrics=['acc'])

### 체크포인트 생성

In [83]:
checkpoint_path = 'my_checkpoint.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss',
                             verbose=1)

### 학습

In [84]:
# 코드입력
model.fit(x_train_padded, train_labels, 
          validation_data=(x_test_padded, test_labels),
          callbacks=[checkpoint],
          epochs=20)

Epoch 1/20
249/250 [============================>.] - ETA: 0s - loss: 0.3533 - acc: 0.8431
Epoch 1: val_loss improved from inf to 0.48649, saving model to my_checkpoint.ckpt
250/250 [==============================] - 25s 46ms/step - loss: 0.3528 - acc: 0.8434 - val_loss: 0.4865 - val_acc: 0.7840
Epoch 2/20
247/250 [============================>.] - ETA: 0s - loss: 0.2684 - acc: 0.8992
Epoch 2: val_loss did not improve from 0.48649
250/250 [==============================] - 4s 18ms/step - loss: 0.2671 - acc: 0.8999 - val_loss: 0.5284 - val_acc: 0.7905
Epoch 3/20
250/250 [==============================] - ETA: 0s - loss: 0.2061 - acc: 0.9345
Epoch 3: val_loss did not improve from 0.48649
250/250 [==============================] - 4s 18ms/step - loss: 0.2061 - acc: 0.9345 - val_loss: 0.4950 - val_acc: 0.7810
Epoch 4/20
250/250 [==============================] - ETA: 0s - loss: 0.1633 - acc: 0.9510
Epoch 4: val_loss did not improve from 0.48649
250/250 [==============================] - 4s

### 체크포인트 로드

In [85]:
# 코드입력
model.load_weights(checkpoint_path)

In [ ]:
 model.save('sample.h5')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
#저장한 모델 불러오기
loaded_model = load_model('sample.h5')

In [ ]:
def run(loaded_model) :
  user_input = input('댓글을 남겨주세요 : ')
  sequence = tokenizer.texts_to ~~

### 결과 예측 (예측 코드는 수정하지 않습니다)

In [86]:
pred = model.predict(test_padded)
your_answer = pred.round().reshape(-1)
your_answer[:5]

array([0., 1., 0., 1., 0.], dtype=float32)

## 결과 제출

### 순서대로 평가가 진행됩니다. 실행은 1회만 해주세요.

- 느리다고 중지 후 다시 평가 코드를 실행하는 경우 제출 과정에서 패널티가 발생할 수 있습니다. (제출 횟수 이슈 발생 가능)

- 전체 점수는 [평가시스템](http://manage.jaen.kr)에서 확인할 수 있습니다.
    - 프로젝트 > 영화리뷰 감정 분석 > 리더보드 > 같은 반 리더보드

아래 Cell을 실행하여 예측 결과 업데이트

In [87]:
# 예측 결과 업데이트
submission = pd.read_csv(os.path.join(DATA_DIR, project_name, 'submission.csv'))
submission['label'] = your_answer
display(submission)

# 제출
project.email = email
project.submit(submission)

,id,label
0,0,0.0
1,1,1.0
2,2,0.0
3,3,1.0
4,4,0.0
...,...,...
9995,9995,1.0
9996,9996,0.0
9997,9997,1.0
9998,9998,0.0


파일을 저장하였습니다. 파일명: submission-07-11-55.csv
제출 여부 :success
오늘 제출 횟수 : 12
제출 결과:0.7779
